In [5]:
import os
import sys
import time

import function_board as fb
import function_tool as ft
import original_code.function_get_aiming_grid as fgag
import original_code.function_evaluate_policy as fep

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import torch
torch.set_printoptions(precision=4)
torch.set_printoptions(linewidth=300)
torch.set_printoptions(threshold=300)

## Noturn Reference

In [150]:
data_parameter_dir = fb.data_parameter_dir
result_dir = './result'  
name_pa = 'player{}'.format(10)  

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

num_aiming_location = aiming_grid.shape[0]
prob_normalscore_1tosmax_dic = {}
prob_normalscore_1tosmaxsum_dic = {}
prob_normalscore_1tosmax_dic_t = {}
prob_normalscore_1tosmaxsum_dic_t = {}
prob_grid_doublescore_dic_t = None
prob_grid_doublescore_dic= None

# Tokens
for score_max in range(0,61):
    score_max_plus1 = score_max + 1 
    prob_normalscore_1tosmax_dic_t[score_max] = np.array(prob_grid_normalscore_t[:,1:score_max_plus1])
    prob_normalscore_1tosmaxsum_dic_t[score_max] = prob_normalscore_1tosmax_dic_t[score_max].sum(axis=1)
if prob_grid_doublescore_dic_t is None:
    prob_doublescore_dic_t = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic_t[doublescore] = np.array(prob_grid_doublescore_t[:,doublescore_index])
else:
    prob_doublescore_dic_t = prob_grid_doublescore_dic_t
prob_DB_t = np.array(prob_grid_bullscore_t[:,1])

# No Tokens

for score_max in range(0,61):
    score_max_plus1 = score_max + 1 
    prob_normalscore_1tosmax_dic[score_max] = np.array(prob_grid_normalscore[:,1:score_max_plus1])
    prob_normalscore_1tosmaxsum_dic[score_max] = prob_normalscore_1tosmax_dic[score_max].sum(axis=1)
if prob_grid_doublescore_dic is None:
    prob_doublescore_dic = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
else:
    prob_doublescore_dic = prob_grid_doublescore_dic
prob_DB = np.array(prob_grid_bullscore[:,1])


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl


In [236]:
## How did probabilities work in the no-turn case? ## 

# Probabilities of all scores from throw actions are zero when using the token probability dictionary 
prob_normalscore_1tosmax_dic_t[60].sum(axis=1)[:740]

# Probabilities of all scores from token actions are zero when using the no-token probability dictionary 
prob_normalscore_1tosmax_dic[60].sum(axis=1)[:740]

# Therefore, they are mutually exclusive

# Cumulative probability of each action across all scores should be one if there are no restrictions on max score

# Max score is a shortcut that allows for easier computation at the other end

# In the noturn case everything is simplified because if you go bust or miss your score stays the same. 
# In the turn case we will need to split this out a bit more 

array([0., 0., 0., ..., 1., 1., 1.])

## Turn Exploration

In [9]:
## single player game without the turn feature
def solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore=None, prob_grid_bullscore=None, prob_grid_doublescore_dic=None):
    """
    Solve the single player game without the turn feature. Find the optimal policy to minimize the expected number of throws for reaching zero score. 
    Args: 
        the action set and the hitting probability associated with the skill model
    
    Returns: 
        optimal_value[score_state]: the expected number of throws for reaching zero from score_state=2,...,501.
        optimal_action_index[score_state]: the index of the aiming location used for score_state=2,...,501.
    """

    num_aiming_location = aiming_grid.shape[0]
    prob_normalscore_1tosmax_dic = {}
    prob_normalscore_1tosmaxsum_dic = {}
    for score_max in range(0,61):
        score_max_plus1 = score_max + 1 
        prob_normalscore_1tosmax_dic[score_max] = np.array(prob_grid_normalscore[:,1:score_max_plus1])
        prob_normalscore_1tosmaxsum_dic[score_max] = prob_normalscore_1tosmax_dic[score_max].sum(axis=1)
    if prob_grid_doublescore_dic is None:
        prob_doublescore_dic = {}
        for doublescore_index in range(20):
            doublescore = 2*(doublescore_index+1)
            prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
    else:
        prob_doublescore_dic = prob_grid_doublescore_dic
    prob_DB = np.array(prob_grid_bullscore[:,1])

    ## possible state: s = 0,1(not possible),2,...,501
    optimal_value = np.zeros(502)
    #optimal_value[1] = np.nan
    optimal_action_index = np.zeros(502, np.int32)
    optimal_action_index[0] = -1
    optimal_action_index[1] = -1
        
    for score_state in range(2,502):            
        ## use matrix operation to search all aiming locations
        
        ## transit to less score state    
        ## s1 = min(score_state-2, 60)
        ## p[z=1]*v[score_state-1] + p[z=2]*v[score_state-2] + ... + p[z=s1]*v[score_state-s1]
        score_max = min(score_state-2, 60)
        score_max_plus1 = score_max + 1 
        ## transit to next state
        num_tothrow = 1.0 + prob_normalscore_1tosmax_dic[score_max].dot(optimal_value[score_state-1:score_state-score_max-1:-1])
        ## probability of transition to state other than s itself
        prob_otherstate = prob_normalscore_1tosmaxsum_dic[score_max]
        
        ## transit to the end of game
        if (score_state == fb.score_DB): ## hit double bull
            prob_otherstate += prob_DB
        elif (score_state <= 40 and score_state%2==0): ## hit double
            prob_otherstate += prob_doublescore_dic[score_state]
        else: ## game does not end
            pass
        
        ## expected number of throw for all aiming locations
        prob_otherstate = np.maximum(prob_otherstate, 0)
        num_tothrow = num_tothrow / prob_otherstate
                            
        ## searching
        optimal_value[score_state] = num_tothrow.min()
        optimal_action_index[score_state] = num_tothrow.argmin()

    return [optimal_value, optimal_action_index]


In [71]:
def init_probabilities(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore):
    
    # aiming grid
    num_aiming_location = aiming_grid.shape[0]
    prob_normalscore = prob_grid_normalscore
    prob_doublescore_dic = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
    prob_DB = np.array(prob_grid_bullscore[:,1])

    ## the probability of not bust for each action given score_max=i (score_remain=i+2)
    prob_bust_dic = {}
    prob_notbust_dic = {}
    for score_max in range(60):    
        ## transit to next throw or turn
        prob_notbust = prob_grid_normalscore[:,0:score_max+1].sum(axis=1)
        ## transit to the end of game
        score_remain = score_max + 2
        if (score_remain == fb.score_DB):
            prob_notbust += prob_DB
        elif (score_remain <= 40 and score_remain%2==0):
            prob_notbust += prob_doublescore_dic[score_remain]
        ##
        prob_notbust = np.minimum(np.maximum(prob_notbust, 0),1)
        prob_notbust_dic[score_max] = prob_notbust
        prob_bust_dic[score_max] = 1 - prob_notbust_dic[score_max]
    
    return num_aiming_location, prob_normalscore, prob_doublescore_dic, prob_DB, prob_bust_dic, prob_notbust_dic

In [80]:
def init_data_structures():
    optimal_value_rt3 = np.zeros(502) #vector: optimal value for the beginning state of each turn (rt=3)
    optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
    optimal_action_index_dic = {}
    num_iteration_record = np.zeros(502, dtype=np.int32)
    
    state_len_vector = np.zeros(4, dtype=np.int32)
    state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
    state_action = [None]  ## aimming locations for for each state in the current playing turn
    action_diff  = [None]
    value_relerror = np.zeros(4)
    for rt in [1,2,3]:
        ## for rt=3: possible score_gained = 0
        ## for rt=2: possible score_gained = 0,1,...,60
        ## for rt=1: possible score_gained = 0,1,...,120
        this_throw_state_len = fb.maxhitscore*(3-rt) + 1
        state_value.append(np.ones(this_throw_state_len)*fb.largenumber)
        state_action.append(np.ones(this_throw_state_len, np.int32)*fb.infeasible_marker)
        action_diff.append(np.ones(this_throw_state_len))
    state_value_update = ft.copy_numberarray_container(state_value)
    state_action_update = ft.copy_numberarray_container(state_action)

    return optimal_value_rt3, optimal_value_dic, optimal_action_index_dic, num_iteration_record, state_len_vector, state_value, state_action, action_diff, value_relerror, state_value_update, state_action_update

In [92]:
def init_score_state(score_state,state_len_vector,state_action,noturn_optimal_action_index,optimal_action_index_dic):     
    ## initialization 
    for rt in [1,2,3]:
        
        ## for rt=3: score_gained = 0
        ## for rt=2: score_gained = 0,1,...,min(s-2,60)
        ## for rt=1: score_gained = 0,1,...,min(s-2,120)
        this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
        state_len_vector[rt] = this_throw_state_len
                
        ## initialize the starting policy: 
        ## use no_turn action in (s, i, u=0)
        ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
        state_action[rt][0] = noturn_optimal_action_index[score_state]            
        for score_gained in range(1,this_throw_state_len):                
            if fb.state_feasible_array[rt, score_gained]:  ## if True
                if fb.state_feasible_array[rt, score_gained-1]:
                    state_action[rt][score_gained] = optimal_action_index_dic[score_state-1][rt][score_gained-1]
                else:                        
                    state_action[rt][score_gained] = noturn_optimal_action_index[score_state-score_gained]
            else:
                state_action[rt][score_gained] = fb.infeasible_marker

    return state_len_vector, state_action

In [87]:
def policy_evaluation(score_state,state_action,prob_grid_normalscore,prob_grid_doublescore,prob_grid_bullscore,prob_bust_dic,optimal_value_rt3,state_value_update): 

    rt = 3
    score_gained = 0
    score_max_turn = min(score_state-2, 3*fb.maxhitscore)

    prob_turn_transit = fep.solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, prob_bust_dic)        
    prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
    new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
    state_value_update[rt][score_gained] = new_value_rt3
    optimal_value_rt3[score_state] = new_value_rt3

    return prob_turn_transit, prob_turn_zeroscore, new_value_rt3
    #print('evaluate rt3 value= {}'.format(new_value_rt3)

9

In [289]:
import copy

def solve_dp_turn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore):
    """
    Solve the single player game with the turn feature.
    Args: 
        the action set and the hitting probability associated with the skill model
    
    Returns: 
        optimal values and the corresponding aiming locations for each state (s,i,u)
    """
    
    ## Initialize probability objects
    num_aiming_location, prob_normalscore, prob_doublescore_dic, prob_DB, prob_bust_dic, prob_notbust_dic = init_probabilities(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    
    prob_normalscore_tensor = torch.from_numpy(prob_normalscore)
    
    iteration_round_limit = 20
    
    # Initialize Data Structures
    optimal_value_rt3, optimal_value_dic, optimal_action_index_dic, num_iteration_record, state_len_vector, state_value, state_action, action_diff, value_relerror, state_value_update, state_action_update = init_data_structures()
    
    ## use no_turn policy as the initial policy
    [noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    
    t1 = time.time()
    for score_state in range(2, 502):
        #print('#### solve_dp_turn score_state={} ####'.format(score_state))    
        
        # Initialize data structures for this score state (i.e. for every number of remaining throws)
        state_len_vector, state_action = init_score_state(score_state,state_len_vector,state_action,noturn_optimal_action_index,optimal_action_index_dic)
    
        ## policy iteration
        for round_index in range(iteration_round_limit):
    
            ## policy evaluation for TURN 
            rt = 3
            score_gained = 0
            score_max_turn = min(score_state-2, 3*fb.maxhitscore)
            # TO DO 
            # pass token probabilities to solve_turn_transit_probability_fast
            # return prob_turn_transit with separate score values for token and non-token
            # "bust" and "finish" should probably be not impacted. 
            prob_turn_transit = fep.solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, prob_bust_dic)        
            # This will be sum of overall bust + score_token[0] + score_notoken[0]
            prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
            
            # This can be staggered like the notoken case 
            # First we add 1 to indicate that we've incremented by a turn 
            # Then we take the dot product for non-token scores with the same token states
            # Then we take the dot product for the token scores with the next token states 

            new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
            # new_value_rt3 = 1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1]) + (prob_turn_zeroscore)*optimal_value_rt3[score_state]

            if score_state==3: 
                temp_action = copy.deepcopy(state_action)
                temp_bust = copy.deepcopy(prob_bust_dic)
                temp_value = copy.deepcopy(optimal_value_rt3)
                temp_max = copy.deepcopy(score_max_turn)

            if score_state==60: 
                print('There was an iteration.')

            # new_value_rt3 = 1 + np.dot 
            state_value_update[rt][score_gained] = new_value_rt3
            optimal_value_rt3[score_state] = new_value_rt3
            #print('evaluate rt3 value= {}'.format(new_value_rt3)
    
            

            ## policy improvement
            for rt in [1,2,3]:            
                this_throw_state_len = state_len_vector[rt]
                
                ## state which can not bust.  score_state-score_gained>=62 
                state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                if (state_notbust_len > 0):
                    if (rt==1 and round_index==0):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len                    
                        next_state_value_array = np.zeros((61, state_notbust_len))                    
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1
                            next_state_value_array[:,score_gained] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                    elif (rt==2 and (round_index==0 or score_state<182)):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len
                        next_state_value_array = np.zeros((61, state_notbust_len))                    
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1
                            next_state_value_array[:,score_gained] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                    else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                        ## only update state of score_gained = 0
                        state_notbust_update_index = 1
                        next_state_value_array = np.zeros(61)
                        score_gained = 0
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1                    
                        ## make a copy
                        if (rt > 1):
                            next_state_value_array[:] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                        ## transit to next turn when rt=1
                        else:
                            next_state_value_array[:] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
    
                    ## matrix product to compute all together
                    next_state_value_tensor = torch.from_numpy(next_state_value_array)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):                    
                        num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                    ## transit to next turn when rt=1
                    else:
                        num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)
    
                    ## searching
                    temp1 = num_turns_tensor.min(axis=0)                
                    state_action_update[rt][0:state_notbust_update_index] = temp1.indices.numpy()
                    state_value_update[rt][0:state_notbust_update_index] =  temp1.values.numpy()                
                
                ## state which possibly bust.  score_state-score_gained<62 
                if (state_notbust_len < this_throw_state_len):
                    ## combine all bust states together 
                    state_bust_len = this_throw_state_len - state_notbust_len
                    next_state_value_array = np.zeros((61, state_bust_len))
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len ## index off set
                        if (rt > 1):
                            next_state_value_array[0:score_max_plus1,score_gained_index] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                        ## transit to next turn when rt=1
                        else:
                            next_state_value_array[0:score_max_plus1,score_gained_index] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                    
                    next_state_value_tensor = torch.from_numpy(next_state_value_array)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):                    
                        num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                    ## transit to next turn when rt=1
                    else:
                        num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)                                                               
    
                    ## consider bust/finishing for each bust state seperately 
                    num_turns_array = num_turns_tensor.numpy()                
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len
    
                        ## transit to the end of game
                        if (rt > 1):
                            if (score_remain == fb.score_DB):                        
                                num_turns_array[:,score_gained_index] += prob_DB
                            elif (score_remain <= 40 and score_remain%2==0):
                                num_turns_array[:,score_gained_index] += prob_doublescore_dic[score_remain]
                            else:
                                pass
    
                        ## transit to bust
                        if (rt==3):
                            num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]
                            ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                            num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max] 
                        elif (rt==2):
                            num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(1+new_value_rt3)
                        else:
                            num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(new_value_rt3)  ## 1 turn is already counted before
    
                    ## searching
                    temp1 = num_turns_tensor.min(axis=0)
                    state_action_update[rt][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                    state_value_update[rt][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                
    
                #### finish rt=1,2,3. check improvement
                action_diff[rt][:] = np.abs(state_action_update[rt] - state_action[rt])                                
                value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
                state_action[rt][:] = state_action_update[rt][:]
                state_value[rt][:] = state_value_update[rt][:]
    
            max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
            max_value_relerror = value_relerror.max()            
            
            if (max_action_diff < 1):
            #if max_value_relerror < iteration_relerror_limit:
                num_iteration_record[score_state] = round_index + 1
                break
    
        for rt in [1,2,3]:
            state_value_update[rt][fb.state_infeasible[rt]] = fb.largenumber
            state_action_update[rt][fb.state_infeasible[rt]] = fb.infeasible_marker
        optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
        optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
        optimal_value_rt3[score_state] = state_value[3][0]
        ## done:V(s,i=3/2/1,u)
    
    ##
    prob_scorestate_transit = {}    
    prob_scorestate_transit =  fep.solve_policy_transit_probability(optimal_action_index_dic, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    t2 = time.time()
    print('solve dp_turn_policyiter in {} seconds'.format(t2-t1))

    print(optimal_value_rt3)
    result_dic = {'optimal_value_dic':optimal_value_dic, 'optimal_action_index_dic':optimal_action_index_dic, 'optimal_value_rt3':optimal_value_rt3, 'prob_scorestate_transit':prob_scorestate_transit}

    return result_dic, temp_action, temp_bust, temp_value, temp_max


In [314]:
[aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore, prob_grid_bullscore] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

result_dic, temp_action, temp_bust, temp_value, temp_max = solve_dp_turn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
There was an iteration.
There was an iteration.


/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/78220377.py:59: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/78220377.py:59: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/714700506.py:185: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/714700506.py:185: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/714700506.py:198: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs

solve prob_policy_transit in 0.2400050163269043 seconds
solve dp_turn_policyiter in 1.8378009796142578 seconds
[0.     0.     1.4381 ... 5.0818 5.0845 5.0965]


In [269]:
temp_action

[None,
 array([40, 55, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32),
 array([40, 54, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32),
 array([40], dtype=int32)]

In [277]:
pt = fep.solve_turn_transit_probability_fast(3, temp_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, temp_bust)        
prob_turn_zeroscore = pt['bust'] + pt['score'][0]
pt

{'finish': 0.6288628603589431,
 'bust': 0.25388779030104686,
 'score': array([6.1874e-13, 1.1725e-01])}

In [292]:
pt['score']#[1:]

array([6.1874e-13, 1.1725e-01])

In [299]:
result_dic['prob_scorestate_transit'][3]

{'finish': 0.6288628603589431,
 'bust': 0.25388779030104686,
 'score': array([6.1874e-13, 1.1725e-01])}

In [286]:
score_state=3

# Their way
new_value_rt3 = (1 + np.dot(pt['score'][1:], temp_value[score_state-1:score_state-temp_max-1:-1])) / (1-prob_turn_zeroscore)
new_value_rt3


1.5662732330740454

In [336]:
# My way
new_value_rt3 = 1 + np.dot(pt['score'][1:], temp_value[score_state-1:score_state-temp_max-1:-1]) + (prob_turn_zeroscore)*temp_value[score_state]
new_value_rt3

# They are not using the old score state - is this some sort of clever trick?

1.567774768324192

In [329]:
# NOT ROUNDING 
# score_state=3

# # Their way
# new_value_rt3 = (1 + np.dot(pt['score'][1:], temp_value[score_state-1:score_state-temp_max-1:-1])) / (1-prob_turn_zeroscore)
# new_value_rt3

# theirs = (1 + round(pt['score'][1:][0],3)*round(temp_value[score_state-1:score_state-temp_max-1:-1][0],3)) / (1 - round(prob_turn_zeroscore,3))

# # My way
# new_value_rt3 = 1 + np.dot(pt['score'][1:], temp_value[score_state-1:score_state-temp_max-1:-1]) + (prob_turn_zeroscore)*temp_value[score_state]
# new_value_rt3

# mine = 1 + round(pt['score'][1:][0],3) * round(temp_value[score_state-1:score_state-temp_max-1:-1][0],3) + round(prob_turn_zeroscore,3) * temp_value[score_state]

# print(theirs)
# print(mine)

1.566013404825737
1.567581600078052


In [333]:
result_dict['optimal_value_rt3'][3]

1.5662732330740456

In [301]:
result_dict['prob_scorestate_transit'][4]['finish'] + result_dict['prob_scorestate_transit'][4]['bust'] + result_dict['prob_scorestate_transit'][4]['score'].sum()

1.0

In [305]:
result_dict['prob_scorestate_transit'][80]['score']

array([0.0000e+00, 0.0000e+00, 0.0000e+00, 5.5489e-70, 2.7641e-88, 1.4717e-17, 1.3926e-04, 4.4390e-05, 6.6158e-18, 1.8069e-08, 1.2698e-04, 1.8989e-07, 2.6002e-04, 1.6312e-04, 8.7872e-08, 1.1314e-05, 2.6983e-05, 2.8800e-04, 1.1336e-05, 2.8645e-04, 5.3083e-06, 3.1907e-04, 1.0591e-03, 9.9865e-05,
       1.2686e-03, 1.4188e-04, 9.2368e-04, 8.7162e-05, 2.4756e-03, 1.3382e-05, 2.6791e-05, 5.7725e-04, 4.3991e-05, 3.2894e-04, 5.5003e-05, 1.7838e-04, 8.5732e-03, 5.2041e-06, 1.4733e-03, 1.7899e-05, 1.5470e-03, 5.3735e-03, 1.9928e-02, 1.4704e-03, 1.5467e-03, 2.0201e-06, 2.5242e-02, 2.2865e-04,
       4.1233e-07, 3.4702e-04, 2.8134e-02, 4.3227e-08, 1.0379e-03, 1.3395e-25, 4.0241e-04, 1.1314e-01, 1.0338e-02, 5.8966e-09, 1.1251e-09, 4.8954e-12, 7.2756e-02, 6.7346e-07, 2.8857e-05, 2.4281e-03, 1.2769e-04, 3.6306e-02, 2.6019e-03, 8.0804e-04, 2.0094e-03, 2.0022e-17, 9.5016e-02, 8.3197e-05,
       1.6277e-07, 1.0243e-03, 3.7020e-04, 2.8711e-02, 7.2032e-03, 8.9433e-08, 9.4250e-09])

In [31]:
result_dict['optimal_value_dic'][4][2].shape


(61,)

In [46]:
result_dict['prob_scorestate_transit'][4]['finish'] + result_dict['prob_scorestate_transit'][4]['bust'] 

0.9245310452865481

In [58]:
result_dict['prob_scorestate_transit'][4]['score'][2]


0.0501178192636838

In [64]:
result_dict['prob_scorestate_transit']

{2: {'finish': 0.6953648488992885,
  'bust': 0.2713529280159355,
  'score': array([0.0333])},
 3: {'finish': 0.6288628603589431,
  'bust': 0.25388779030104686,
  'score': array([6.1874e-13, 1.1725e-01])},
 4: {'finish': 0.8261802527229658,
  'bust': 0.09835079256358226,
  'score': array([2.5351e-002, 4.3619e-178, 5.0118e-002])},
 5: {'finish': 0.7189990957602209,
  'bust': 0.11385574051212324,
  'score': array([6.1874e-13, 7.1915e-02, 5.0438e-12, 9.5230e-02])},
 6: {'finish': 0.7102429963383129,
  'bust': 0.12553971598927383,
  'score': array([5.8417e-008, 8.5085e-155, 7.0208e-002, 4.7075e-008, 9.4009e-002])},
 7: {'finish': 0.7178426161948032,
  'bust': 0.11234823998304272,
  'score': array([5.8884e-007, 1.0646e-124, 8.8248e-016, 7.3742e-002, 7.3127e-017, 9.6067e-002])},
 8: {'finish': 0.7425290013447942,
  'bust': 0.055721671289694845,
  'score': array([3.6063e-02, 2.3973e-19, 3.7320e-29, 3.1871e-30, 9.7742e-02, 2.0744e-39, 6.7944e-02])},
 9: {'finish': 0.6956863004298276,
  'bust': 